# <h1 style="color:red;">Initial Setup</h>

Import all necessary modules

In [ ]:
import nest_asyncio
import asyncio
import schwabdev
from datetime import datetime, timedelta
from dotenv import load_dotenv
import time
import os
import json
import pandas as pd
import pytz
import numpy as np
import mplfinance as mpf
from IPython.display import display, clear_output
import warnings
from Multi_Strat_Back_Tester_Functions import *
from datetime import datetime, time
import matplotlib.pyplot as plt
from itertools import product

Load environment variables from .env file for authentification purposes

In [ ]:
load_dotenv()

True

Patches the notebook's running event loop so that it can handle multiple calls to `asyncio.run()` or other asynchronous methods without causing conflicts

In [ ]:
nest_asyncio.apply()

Set the display options

In [ ]:
# Display options
# Set the global float format to 4 decimal places
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
pd.options.display.float_format = '{:,.4f}'.format # Format numerical output to have certain number of decimals
# pd.options.display.float_format = None # Reset to default numerical output formatting
pd.set_option('display.width', 2000) # Set the display width to a large number
pd.set_option('display.max_colwidth', 1000) # Set max column width to a large number

pd.set_option('display.max_columns', None) # Displays all columns
# pd.set_option('display.max_rows', None) # Displays all rows                             
# pd.reset_option('display.max_columns') # Display default abbreviated columns
pd.reset_option('display.max_rows') # Display default abbreviated rows

Create the client object.

In [ ]:
app_key = os.getenv('app_key')
app_secret = os.getenv('app_secret')
callback_url = os.getenv('callback_url')

# Print them to verify (avoid printing sensitive info in production)
print(f"App Key: {app_key}")
print(f"App Secret: {app_secret}")
print(f"Callback URL: {callback_url}")

# Now proceed to initialize the client
client = schwabdev.Client(app_key, app_secret, callback_url)
print('Client initialized!')

App Key: 2C7Jh6zt5QdlSb0N5RnhWpaiEzKFr150
App Secret: OQGxH0GkD5bAEMA4
Callback URL: https://127.0.0.1
The refresh token has expired, please update!
Open to authenticate: https://api.schwabapi.com/v1/oauth/authorize?client_id=2C7Jh6zt5QdlSb0N5RnhWpaiEzKFr150&redirect_uri=https://127.0.0.1
Client initialized!


Use this when in need of updating the refresh token.

In [ ]:
# client.update_tokens(force=True)
# client = schwabdev.Client(app_key, app_secret, callback_url)

# <h1 style="color:red;">Permanent Data Extraction and Storage Functions</h>

Ticker list and point value definition.

In [ ]:
tickers = "/ES,/NQ,/CL,/GC"
# tickers = "MARA,PLUG,SOFI,SWN,RKLB"

ticker_to_point_value = {
    "/ES": 50,       # E-Mini S&P 500
    "/NQ": 20,       # E-Mini NASDAQ 100
    "/CL": 1000,     # Crude Oil
    "/GC": 100,      # Gold
}

ticker_to_tick_size = {
    "/ES": 0.25,        # E-Mini S&P 500
    "/NQ": 0.25,        # E-Mini NASDAQ 100
    "/CL": 0.01,        # Crude Oil
    "/GC": 0.1,         # Gold
}

tick_size = ticker_to_tick_size.get("/ES")
tick_size

0.25

In [ ]:
# tickers = "/ES,/NQ,/CL,/GC,/MES,/MNQ,/MCL,/MGC"
# # tickers = "MARA,PLUG,SOFI,SWN,RKLB"

# ticker_to_point_value = {
#     "/ES": 50,       # E-Mini S&P 500
#     "/NQ": 20,       # E-Mini NASDAQ 100
#     "/CL": 1000,     # Crude Oil
#     "/GC": 100,      # Gold
#     "/MES": 5,       # Micro E-Mini S&P 500
#     "/MNQ": 2,       # Micro E-Mini NASDAQ 100
#     "/MCL": 100,     # Micro Crude Oil
#     "/MGC": 10       # Micro Gold
# }

# ticker_to_tick_size = {
#     "/ES": 0.25,        # E-Mini S&P 500
#     "/NQ": 0.25,        # E-Mini NASDAQ 100
#     "/CL": 0.01,        # Crude Oil
#     "/GC": 0.1,         # Gold
#     "/MES": 0.25,       # Micro E-Mini S&P 500
#     "/MNQ": 0.25,       # Micro E-Mini NASDAQ 100
#     "/MCL": 0.01,       # Micro Crude Oil
#     "/MGC": 0.1         # Micro Gold
# }

# tick_size = ticker_to_tick_size.get("/ES")
# tick_size

Extracting and storing raw streaming data with two functions into ticker_tables dictionary. 

In [ ]:
def extract_row_data(entry, content):
    
    # Create a row dictionary with the relevant fields
    row = { # For futures
        'timestamp': entry['timestamp'],
        'key': content.get('key', None),
        'bid_price': content.get('1', None),
        'ask_price': content.get('2', None),
        # 'last_price': last_price,  # Use updated last_price
        'last_price': content.get('3', None),
        'bid_size': content.get('4', None), # These two variables can be correlated 
        'ask_size': content.get('5', None), # with direction of price action
        'total_volume': content.get('8', None),
        'last_size': content.get('9', 0),
        'high_price': content.get('12', None),
        'low_price': content.get('13', None),
        'close_price': content.get('14', None),
        'open_price': content.get('18', None),
        'net_change': content.get('19', None),
        'future_pct_change': content.get('20', None),
        'open_interest': content.get('23', None),
        'tick': content.get('25', None),
        'tick_amount': content.get('26', None),
        'future_exp_date': content.get('35', None),
        'ask_time': content.get('37', None),
        'bid_time': content.get('38', None)
        }

    # row = { # For equities
    #     'timestamp': entry['timestamp'],               # Same
    #     'key': content.get('key', None),               # Same
    #     'bid_price': content.get('1', None),           # Matches bid_price
    #     'ask_price': content.get('2', None),           # Matches ask_price
    #     'last_price': last_price,                      # Matches last_price (updated)
    #     'bid_size': content.get('4', None),            # Matches bid_size
    #     'ask_size': content.get('5', None),            # Matches ask_size
    #     'total_volume': content.get('8', None),        # Matches total_volume
    #     'last_size': content.get('9', 0),              # Matches last_size
    #     'high_price': content.get('10', None),         # Matches high_price
    #     'low_price': content.get('11', None),          # Matches low_price
    #     'close_price': content.get('12', None),        # Matches close_price
    #     'open_price': content.get('17', None),         # Matches open_price
    #     'net_change': content.get('18', None),         # Matches net_change
    #     '52_week_high': content.get('19', None),       # Matches 52_week_high
    #     '52_week_low': content.get('20', None),        # Matches 52_week_low
    #     'pe_ratio': content.get('21', None),           # Matches pe_ratio
    #     'net_pct_change': content.get('42', None)}      # Matches net_pct_change  
    
    return row

In [ ]:
ticker_tables = 'nothing'
ticker_tables = {ticker: pd.DataFrame() for ticker in tickers.split(",")}

# Initialize a dictionary to store the last known price for each ticker
last_known_price = {ticker: None for ticker in tickers.split(",")}

def process_message_data(data):
    global ticker_tables, last_known_price
    
    # Loop through the content and extract the relevant fields
    if "data" in data:
        for entry in data['data']:
            if 'content' in entry:
                for content in entry['content']:
                    # Extract row data
                    row = extract_row_data(entry, content)

                    # Get the ticker (key) and last_price
                    ticker = row['key']
                    last_price = row.get('last_price')

                    # Check if last_price is None, if so use the last known price for the ticker
                    if last_price is None and ticker in last_known_price:
                        row['last_price'] = last_known_price[ticker]
                    else:
                        # Update the last known price
                        last_known_price[ticker] = last_price

                    # Convert the row dictionary to a DataFrame
                    row_df = pd.DataFrame([row])

                    # Check the ticker and append the row to the correct ticker table
                    if ticker in ticker_tables:
                        ticker_tables[ticker] = pd.concat([ticker_tables[ticker], row_df], ignore_index=True)

Compressing minute candles from the raw data.

In [ ]:
minute_candles = 'nothing'
minute_candles = {ticker: pd.DataFrame(columns=['datetime', 'ticker', 'open', 'high', 'low', 'close', 'accumulative_volume']) for ticker in tickers.split(",")}

def update_minute_candles(ticker_tables, time_frame='min'):
    global minute_candles

    for ticker, df in ticker_tables.items():
        if df.empty:
            continue
        
        # Ensure that 'datetime' column exists and is properly set
        if 'timestamp' in df.columns:
            df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True).dt.tz_convert('US/Eastern')

        # Ensure that the 'datetime' column is set as the DataFrame index (only if not already set)
        if df.index.name != 'datetime':
            df.set_index('datetime', inplace=True)

        # Group the data by minute intervals using 'min'
        current_minute = df.index.floor(time_frame)[-1]  # Get the most recent minute

        # Select only the rows from the most recent minute
        recent_data = df[df.index.floor(time_frame) == current_minute]

        if not recent_data.empty:
            # Extract OHLC values for the minute candle
            open_price = recent_data['last_price'].iloc[0]  # Open is the first last_price
            high_price = recent_data['last_price'].max()     # High is the max last_price
            low_price = recent_data['last_price'].min()      # Low is the min last_price
            close_price = recent_data['last_price'].iloc[-1] # Close is the last last_price
            total_volume = recent_data['total_volume'].max() # Total volume is the max value in this interval

            # Create a dictionary with the new candle data
            new_candle = {
                'datetime': current_minute,
                'ticker': ticker,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'accumulative_volume': total_volume
            }

            # Convert the dictionary to a DataFrame and update the respective ticker's DataFrame in minute_candles
            new_candle_df = pd.DataFrame([new_candle])

            # Replace or append the most recent candle for this ticker
            if not minute_candles[ticker].empty and minute_candles[ticker]['datetime'].iloc[-1] == current_minute:
                minute_candles[ticker].iloc[-1] = new_candle  # Update the last candle
            else:
                minute_candles[ticker] = pd.concat([minute_candles[ticker], new_candle_df], ignore_index=True)  # Append new candle


In [ ]:
# Create the new nested dictionary
compressed_sessions = {}

def compress_candles(candles, n):
    """
    Compress candles for a given timeframe across a DataFrame.
    Preserves time-derived columns if they exist.

    Parameters:
    - candles (pd.DataFrame): DataFrame containing the candle data.
    - n (int): Number of minutes (rows) to compress into one candle.

    Returns:
    - pd.DataFrame: A compressed DataFrame.
    """
    if not isinstance(candles, pd.DataFrame) or n <= 0:
        raise ValueError("Input must be a DataFrame, and 'n' must be a positive integer.")

    # Ensure sorted
    candles = candles.sort_values('datetime').reset_index(drop=True)

    # Group every n rows
    grouped = candles.groupby(candles.index // n)

    # Columns to always aggregate
    agg_dict = {
        'datetime': 'first',
        'ticker': 'first',
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'accumulative_volume': 'sum'
    }

    # Optionally preserve time-derived columns
    for col in ['year', 'month', 'week', 'day', 'weekday', 'hour', 'minute']:
        if col in candles.columns:
            agg_dict[col] = 'first'

    compressed = grouped.agg(agg_dict).reset_index(drop=True)

    return compressed

A function that calculates a number of indicators.

In [ ]:
def calculate_indicators_1(
    df, 
    price_col='close', 
    acc_vol_col='accumulative_volume', 
    sma_periods=None, 
    wma_periods=None, 
    rsi_periods=None, 
    volume_col='volume', 
    candle_window=10, 
    base_window=5, 
    adaptive_window=3,
    trend_window=6
):
    """
    Calculate technical indicators, including SMAs, WMAs, RSI, price action envelopes, and trend score.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame containing the price data.
    - price_col (str): The column name for the price data (default 'close').
    - acc_vol_col (str): The column name for accumulative volume data (default 'accumulative_volume').
    - sma_periods (list): A list of periods for SMAs.
    - wma_periods (list): A list of periods for WMAs.
    - rsi_periods (list): A list of periods for RSI.
    - volume_col (str): The column name to store the volume difference (default 'volume').
    - candle_window (int): The window size for candle metrics calculations (default 10).
    - base_window (int): The window size for detecting local highs/lows in the price action envelope.
    - adaptive_window (int): The alternative smaller window size for adaptive price action envelope.
    - trend_window (int): The window size for calculating trend scores.

    Returns:
    - pd.DataFrame: The updated DataFrame with calculated indicators.
    """

    # Ensure RSI periods is a valid list
    if rsi_periods is None:
        rsi_periods = []

    # Calculate volume differences
    df[volume_col] = df[acc_vol_col].diff()

    # Calculate SMAs
    if sma_periods:
        for period in sma_periods:
            df[f'sma_{period}'] = df[price_col].rolling(window=period).mean()

    # Calculate WMAs
    if wma_periods:
        for period in wma_periods:
            weights = np.arange(1, period + 1)  # Generate weights from 1 to the period length
            df[f'wma_{period}'] = df[price_col].rolling(window=period).apply(
                lambda prices: np.dot(prices, weights) / weights.sum(), raw=True
            )

    # calculate_vwap_and_bands(df, high_col='high', low_col='low', close_col='close', volume_col='volume')

    # Calculate RSI
    for period in rsi_periods:
        delta = df[price_col].diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
        rs = gain / loss
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))

    # # Calculate price action envelope (Updated with dynamic base_window & adaptive_window)
    # df = calculate_price_action_envelope_1(df, high_col='high', low_col='low', base_window=base_window, adaptive_window=adaptive_window)

    # # Calculate trend score (Added)
    # df['trend_indicator'] = calculate_trend_score_1(df, open_col='open', high_col='high', low_col='low', close_col='close', window=trend_window)

    # Calculate additional candle properties
    df['ohlc_average'] = df[['open', 'high', 'low', 'close']].mean(axis=1)
    df['candle_span'] = df['high'] - df['low']
    df['candle_body'] = (df['close'] - df['open']).abs()
    df['candle_span_avg'] = df['candle_span'].rolling(window=candle_window, min_periods=1).mean()
    df['candle_span_max'] = df['candle_span'].rolling(window=candle_window, min_periods=1).max()
    df['candle_span_maxavg_mean'] = (df['candle_span_avg'] + df['candle_span_max']) / 2

    # Add constant reference lines for trend analysis
    df['hundred_line'] = 100
    df['fifty_line'] = 50
    df['zero_line'] = 0
    df['trend_high_threshold'] = 75
    df['trend_low_threshold'] = 25

    return df

# <h1 style="color:red;">Trade Logic</h>

In [ ]:
def generate_trading_signals_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Generate buy/sell signals based on moving averages and RSI indicators for either
    a trend-following or mean-reversion strategy.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - strategy_type (str): 'trend' or 'reversion'.

    Returns:
    - candles (pd.DataFrame): The DataFrame with updated signal and position state columns.
    """

    # Append strategy type to column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_signal_column = f'signal_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_position_open_col = f'position_open_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_position_open_col = f'position_open_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Initialize columns
    candles[ma_signal_column] = 0
    candles[rsi_signal_column] = 0
    candles[ma_position_open_col] = False
    candles[rsi_position_open_col] = False

    # ======= MOVING AVERAGE LOGIC =======
    ma_position_open = True
    ma_signals = []
    ma_positions = []

    for ma1, ma2 in zip(candles[ma_name1], candles[ma_name2]):
        if strategy_type == 'trend':
            # Trend-following logic
            if ma_position_open and ma1 <= ma2: # if position is neutral and wma is below sma
                ma_position_open = False
                ma_signals.append(0)  # Neutral
            elif not ma_position_open and ma1 > ma2:
                ma_position_open = True
                ma_signals.append(1)  # Long
            else:
                ma_signals.append(ma_signals[-1] if ma_signals else 0)
        elif strategy_type == 'reversion':
            # Mean-reversion logic (inverted)
            if ma_position_open and ma1 > ma2:
                ma_position_open = False
                ma_signals.append(0)  # Neutral
            elif not ma_position_open and ma1 <= ma2:
                ma_position_open = True
                ma_signals.append(1)  # Long
            else:
                ma_signals.append(ma_signals[-1] if ma_signals else 0)

        ma_positions.append(ma_position_open)

    candles[ma_signal_column] = ma_signals
    candles[ma_position_open_col] = ma_positions

    # ======= RSI LOGIC =======
    rsi_position_open = True
    rsi_signals = []
    rsi_positions = []

    for rsi in candles[rsi_column]:
        if strategy_type == 'trend':
            if rsi_position_open and rsi < 50:
                rsi_position_open = False
                rsi_signals.append(0)  # Neutral
            elif not rsi_position_open and rsi >= 50:
                rsi_position_open = True
                rsi_signals.append(1)  # Long
            else:
                rsi_signals.append(rsi_signals[-1] if rsi_signals else 0)
        elif strategy_type == 'reversion':
            if rsi_position_open and rsi >= 50:
                rsi_position_open = False
                rsi_signals.append(0)  # Neutral
            elif not rsi_position_open and rsi < 50:
                rsi_position_open = True
                rsi_signals.append(1)  # Long
            else:
                rsi_signals.append(rsi_signals[-1] if rsi_signals else 0)

        rsi_positions.append(rsi_position_open)

    candles[rsi_signal_column] = rsi_signals
    candles[rsi_position_open_col] = rsi_positions

    return candles

def update_position_open_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Update the 'position_open' columns for MA and RSI strategies for a specific strategy type (trend or reversion).

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing the signals and position columns.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI signal.
    - strategy_type (str): Either 'trend' or 'reversion'

    Returns:
    - pd.DataFrame: The updated DataFrame with 'position_open' columns for the chosen strategy type.
    """
    # Append strategy type to column names
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_signal_column = f'signal_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_position_open = f'position_open_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_position_open = f'position_open_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    
    # Update position open columns based on the signals
    candles[ma_position_open] = candles[ma_signal_column] == 1
    candles[rsi_position_open] = candles[rsi_signal_column] == 1
    
    return candles

def determine_entry_prices_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', 
                             ticker_to_tick_size=None, ticker=None, strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Determine entry prices for MA and RSI strategies based on signals.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - ticker_to_tick_size (dict): Mapping of tickers to their tick sizes.
    - ticker (str): The ticker for which the tick size applies.
    - strategy_type (str): Either 'trend' or 'reversion', affecting the signal logic.
    - order_type (str): 'market' or 'limit', defining how entry prices are set.

    Returns:
    - candles (pd.DataFrame): The DataFrame with updated entry price columns.
    """
    # Dynamically generate column names including the strategy type
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_signal_column = f'signal_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_entry_price = f'entry_price_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_entry_price = f'entry_price_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Initialize entry price columns
    candles[ma_entry_price] = None
    candles[rsi_entry_price] = None

    # Get tick size
    tick_size = ticker_to_tick_size.get(ticker, 0) if ticker_to_tick_size else 0

    # Determine tick size adjustment based on order type
    add_tick_size = order_type == 'market'

    # Moving Average Strategy
    ma_signals = candles[ma_signal_column]
    ma_close_prices = candles['close']
    ma_entry_mask = (ma_signals == 1) & (ma_signals.shift(1) != 1)
    candles.loc[ma_entry_mask, ma_entry_price] = ma_close_prices[ma_entry_mask] + (tick_size if add_tick_size else 0)

    # RSI Strategy
    rsi_signals = candles[rsi_signal_column]
    rsi_entry_mask = (rsi_signals == 1) & (rsi_signals.shift(1) != 1)
    candles.loc[rsi_entry_mask, rsi_entry_price] = ma_close_prices[rsi_entry_mask] + (tick_size if add_tick_size else 0)

    return candles

def determine_exit_prices_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', 
                            ticker_to_tick_size=None, ticker=None, strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Determine exit prices for MA and RSI strategies based on signals.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - ticker_to_tick_size (dict): Mapping of tickers to their tick sizes.
    - ticker (str): The ticker for which the tick size applies.
    - strategy_type (str): Either 'trend' or 'reversion', affecting the signal logic.
    - order_type (str): 'market' or 'limit', defining how exit prices are set.

    Returns:
    - candles (pd.DataFrame): The DataFrame with updated exit price columns.
    """
    # Dynamically generate column names including the strategy type
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_signal_column = f'signal_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_exit_price = f'exit_price_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_exit_price = f'exit_price_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Initialize exit price columns
    candles[ma_exit_price] = None
    candles[rsi_exit_price] = None

    # Get tick size
    tick_size = ticker_to_tick_size.get(ticker, 0) if ticker_to_tick_size else 0

    # Determine tick size adjustment based on order type
    subtract_tick_size = order_type == 'market'

    # Moving Average Strategy
    ma_signals = candles[ma_signal_column]
    ma_close_prices = candles['close']
    ma_exit_mask = (ma_signals == 0) & (ma_signals.shift(1) == 1)
    candles.loc[ma_exit_mask, ma_exit_price] = ma_close_prices[ma_exit_mask] - (tick_size if subtract_tick_size else 0)

    # RSI Strategy
    rsi_signals = candles[rsi_signal_column]
    rsi_exit_mask = (rsi_signals == 0) & (rsi_signals.shift(1) == 1)
    candles.loc[rsi_exit_mask, rsi_exit_price] = ma_close_prices[rsi_exit_mask] - (tick_size if subtract_tick_size else 0)

    return candles

def calculate_stop_losses_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Dynamically calculate stop loss levels for MA and RSI strategies and ensure they persist while positions are open.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - strategy_type (str): Either 'trend' or 'reversion', affecting column naming.

    Returns:
    - pd.DataFrame: The updated DataFrame with dynamically named stop loss columns.
    """
    # Dynamically generate column names including the strategy type
    ma_entry_price = f'entry_price_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    ma_exit_price = f'exit_price_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_entry_price = f'entry_price_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_exit_price = f'exit_price_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    stop_loss_ma = f'stop_loss_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    stop_loss_rsi = f'stop_loss_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Initialize stop loss columns
    candles[stop_loss_ma] = None
    candles[stop_loss_rsi] = None

    # Moving Average Stop Loss
    ma_entry_mask = candles[ma_entry_price].notnull()
    ma_exit_mask = candles[ma_exit_price].notnull()
    
    # Set stop loss where positions open
    candles.loc[ma_entry_mask, stop_loss_ma] = candles[ma_entry_price] - candles['candle_span_max']

    # Reset stop loss and close position where positions close
    candles.loc[ma_exit_mask, stop_loss_ma] = None

    # RSI Stop Loss
    rsi_entry_mask = candles[rsi_entry_price].notnull()
    rsi_exit_mask = candles[rsi_exit_price].notnull()
    
    # Set stop loss where positions open
    candles.loc[rsi_entry_mask, stop_loss_rsi] = candles[rsi_entry_price] - candles['candle_span_max']

    # Reset stop loss and close position where positions close
    candles.loc[rsi_exit_mask, stop_loss_rsi] = None

    # Forward-fill stop loss for both strategies
    candles[stop_loss_ma] = candles[stop_loss_ma].ffill()
    candles[stop_loss_rsi] = candles[stop_loss_rsi].ffill()

    return candles

def track_stop_loss_hits_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', ticker_to_tick_size=None, ticker=None, strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Track whether stop losses have been hit for MA and RSI strategies and update dynamically named columns.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - ticker_to_tick_size (dict): Mapping of tickers to their tick sizes.
    - ticker (str): The ticker for which the tick size applies.
    - strategy_type (str): Either 'trend' or 'reversion', affecting column naming.

    Returns:
    - pd.DataFrame: The updated DataFrame with dynamically named stop loss hit flags.
    """
    # Dynamically generate column names including strategy type
    stop_loss_ma = f'stop_loss_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    stop_loss_rsi = f'stop_loss_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_stop_loss_hit = f'stop_loss_hit_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_stop_loss_hit = f'stop_loss_hit_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_position_open = f'position_open_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_position_open = f'position_open_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Initialize stop loss hit columns
    candles[ma_stop_loss_hit] = False
    candles[rsi_stop_loss_hit] = False

    # Get tick size (ensure it's non-zero)
    tick_size = ticker_to_tick_size.get(ticker, 0) if ticker_to_tick_size else 0

    # Ensure stop loss values are numerical (convert None to NaN)
    candles[stop_loss_ma] = candles[stop_loss_ma].fillna(float('inf'))
    candles[stop_loss_rsi] = candles[stop_loss_rsi].fillna(float('inf'))

    # Moving Average Stop Loss Hit Logic
    ma_hit_condition = (
        (candles[stop_loss_ma].notnull()) & 
        (candles['close'] <= (candles[stop_loss_ma])) & 
        candles[ma_position_open]
    )
    candles.loc[ma_hit_condition, ma_stop_loss_hit] = True

    # RSI Stop Loss Hit Logic
    rsi_hit_condition = (
        (candles[stop_loss_rsi].notnull()) & 
        (candles['close'] <= (candles[stop_loss_rsi])) & 
        candles[rsi_position_open]
    )
    candles.loc[rsi_hit_condition, rsi_stop_loss_hit] = True

    return candles

def adjust_signals_for_stop_loss_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Adjust MA and RSI signals to 0 where stop loss has been hit.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - strategy_type (str): Either 'trend' or 'reversion', affecting column naming.

    Returns:
    - pd.DataFrame: The updated DataFrame with adjusted signals.
    """
    # Dynamically generate column names including strategy type
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_signal_column = f'signal_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_stop_loss_hit_column = f'stop_loss_hit_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_stop_loss_hit_column = f'stop_loss_hit_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Adjust MA and RSI signals where stop loss has been hit
    candles.loc[candles[ma_stop_loss_hit_column], ma_signal_column] = 0
    candles.loc[candles[rsi_stop_loss_hit_column], rsi_signal_column] = 0

    return candles

def update_stop_loss_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', strategy_type='trend', order_type='market', directional_bias='long'):
    """
    Dynamically set stop loss columns to NaN where corresponding signal columns are 0.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - strategy_type (str): Either 'trend' or 'reversion', affecting column naming.

    Returns:
    - pd.DataFrame: The updated DataFrame with dynamically named stop loss columns modified.
    """
    # Dynamically generate column names including strategy type
    ma_signal_column = f'signal_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_signal_column = f'signal_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    stop_loss_ma = f'stop_loss_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    stop_loss_rsi = f'stop_loss_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Update stop loss columns to NaN where signals are 0
    candles.loc[candles[ma_signal_column] == 0, stop_loss_ma] = float('nan')
    candles.loc[candles[rsi_signal_column] == 0, stop_loss_rsi] = float('nan')

    return candles

def calculate_profit_loss_1(candles, ma_name1='wma_5', ma_name2='sma_5', rsi_column='rsi_5', contract_multiplier=1, trade_commission=1.5, strategy_type='trend', order_type='market', directional_bias='long'):
    """ 
    Dynamically calculate profit and loss based on entry and exit price columns, including cumulative commission costs.

    Parameters:
    - candles (pd.DataFrame): The DataFrame containing candle data with entry and exit price columns.
    - contract_multiplier (float): The multiplier for PnL calculation (e.g., contract size).
    - trade_commission (float): The commission cost per trade.
    - ma_name1 (str): Column name for the first moving average.
    - ma_name2 (str): Column name for the second moving average.
    - rsi_column (str): Column name for the RSI indicator.
    - strategy_type (str): The type of strategy ('trend' or 'reversion').

    Returns:
    - pd.DataFrame: The DataFrame with dynamically named profit/loss and commission cost columns.
    """
    # Dynamically generate column names with strategy type
    ma_entry_price = f'entry_price_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    ma_exit_price = f'exit_price_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_entry_price = f'entry_price_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_exit_price = f'exit_price_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    pnl_ma_col = f'pnl_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    pnl_rsi_col = f'pnl_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    cum_pnl_ma_col = f'cum_{pnl_ma_col}'
    cum_pnl_rsi_col = f'cum_{pnl_rsi_col}'
    cum_pnl_all_col = f'cum_pnl_all_{ma_name1}_{ma_name2}_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'
    ma_commission_col = f'commission_cost_{ma_name1}_{ma_name2}_{strategy_type}_{order_type}_{directional_bias}'
    rsi_commission_col = f'commission_cost_{rsi_column}_{strategy_type}_{order_type}_{directional_bias}'

    # Initialize PnL and commission columns
    candles[pnl_ma_col] = 0.0
    candles[pnl_rsi_col] = 0.0
    candles[ma_commission_col] = 0.0
    candles[rsi_commission_col] = 0.0

    # Moving Average Strategy PnL and Commission Calculation
    ma_entry_indices = candles.index[candles[ma_entry_price].notnull()]
    ma_exit_indices = candles.index[candles[ma_exit_price].notnull()]

    # Pair up entry and exit prices
    valid_pairs_ma = min(len(ma_entry_indices), len(ma_exit_indices))
    ma_entry_prices = candles.loc[ma_entry_indices[:valid_pairs_ma], ma_entry_price].values
    ma_exit_prices = candles.loc[ma_exit_indices[:valid_pairs_ma], ma_exit_price].values

    # Calculate commission costs for MA strategy
    candles[ma_commission_col] = candles[ma_entry_price].notna().astype(int) * trade_commission + \
                                 candles[ma_exit_price].notna().astype(int) * trade_commission
    candles[ma_commission_col] = candles[ma_commission_col].cumsum()  # Accumulate commission costs

    # Calculate PnL for MA strategy
    ma_pnl = (ma_exit_prices - ma_entry_prices) * contract_multiplier
    candles.loc[ma_exit_indices[:valid_pairs_ma], pnl_ma_col] = ma_pnl

    # RSI Strategy PnL and Commission Calculation
    rsi_entry_indices = candles.index[candles[rsi_entry_price].notnull()]
    rsi_exit_indices = candles.index[candles[rsi_exit_price].notnull()]

    # Pair up entry and exit prices
    valid_pairs_rsi = min(len(rsi_entry_indices), len(rsi_exit_indices))
    rsi_entry_prices = candles.loc[rsi_entry_indices[:valid_pairs_rsi], rsi_entry_price].values
    rsi_exit_prices = candles.loc[rsi_exit_indices[:valid_pairs_rsi], rsi_exit_price].values

    # Calculate commission costs for RSI strategy
    candles[rsi_commission_col] = candles[rsi_entry_price].notna().astype(int) * trade_commission + \
                                  candles[rsi_exit_price].notna().astype(int) * trade_commission
    candles[rsi_commission_col] = candles[rsi_commission_col].cumsum()  # Accumulate commission costs

    # Calculate PnL for RSI strategy
    rsi_pnl = (rsi_exit_prices - rsi_entry_prices) * contract_multiplier
    candles.loc[rsi_exit_indices[:valid_pairs_rsi], pnl_rsi_col] = rsi_pnl

    # Calculate cumulative PnL for both strategies
    candles[cum_pnl_ma_col] = candles[pnl_ma_col].cumsum()
    candles[cum_pnl_rsi_col] = candles[pnl_rsi_col].cumsum()

    # Calculate combined cumulative PnL
    candles[cum_pnl_all_col] = candles[cum_pnl_ma_col] + candles[cum_pnl_rsi_col]

    return candles

In [ ]:
# # Initialize a variable to track if a position is open (market neutral) for function `trade_logic`
# position_open = False  # False means market neutral, True means a trade is active
# entry_price = None  # Store the entry price when a trade is opened

# # Initialize a variable to track if a position is open (market neutral) for function `trade_logic`
# sim_position_open = False  # False means market neutral, True means a trade is active
# sim_entry_price = None  # Store the entry price when a trade is opened

# def trade_logic(candles, buy_order=None, sell_order=None):
#     """
#     Function to decide whether to go long or close positions based on the signal column.

#     Parameters:
#     - candles (pd.DataFrame): The DataFrame containing minute candle data.
#     - buy_order (dict): The buy order configuration (market order).
#     - sell_order (dict): The sell order configuration (market order).
#     """

#     global position_open, entry_price  # Ensure access to global variables

#     current_hour = datetime.now(pytz.timezone('US/Eastern')).hour
#     current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
#     current_second = datetime.now(pytz.timezone('US/Eastern')).second

#     # Check if it's the start of a new minute (second == 00)
#     if current_second == 0 or 1:
#         # Ensure there is enough data in candles (at least 2 rows to check the previous signal)
#         if len(candles) < 2:
#             print("Not enough data to make a trade decision.")
#             return

#         # Get the signal from the row before the one currently forming
#         previous_signal = candles.iloc[-2]['signal']

#         if previous_signal == 1:
#             # If signal is 1 and no position is open, go long (buy)
#             if not position_open:
#                 # Place buy order
#                 resp = client.order_place(account_hash, buy_order)
#                 print(f"Buy order placed. Response: {resp}")
                
#                 # Mark position as open and record entry price
#                 position_open = True
#                 entry_price = candles.iloc[-2]['close']
#                 print(f"Bought at {entry_price}")
#             else:
#                 print("Position already open, no action taken.")

#         elif previous_signal == 0 or previous_signal == -1:
#             # If signal is 0 or -1 and a position is open, close the position (sell)
#             if position_open:
#                 # Place sell order
#                 resp = client.order_place(account_hash, sell_order)
#                 print(f"Sell order placed. Response: {resp}")
                
#                 # Close position
#                 position_open = False
#                 print(f"Sold at {candles.iloc[-2]['close']}")
#             else:
#                 print("No position open, no action taken.")

# def trade_simulator(candles):
#     """
#     Simulated function to decide whether to go long or close positions based on the signal column.

#     Parameters:
#     - candles (pd.DataFrame): The DataFrame containing minute candle data.

#     The function updates the 'sim_signal', 'sim_entry', and 'sim_exit' columns in the DataFrame.
#     """

#     global sim_position_open, sim_entry_price  # Ensure access to global variables

#     # Initialize the new columns if they don't exist
#     if 'sim_signal' not in candles.columns:
#         candles['sim_signal'] = None
#     if 'sim_entry' not in candles.columns:
#         candles['sim_entry'] = None
#     if 'sim_exit' not in candles.columns:
#         candles['sim_exit'] = None

#     current_hour = datetime.now(pytz.timezone('US/Eastern')).hour
#     current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
#     current_second = datetime.now(pytz.timezone('US/Eastern')).second

#     # Check if it's the start of a new minute (second == 00)
#     if current_second == 0 or 1:
#         # Ensure there is enough data in candles (at least 2 rows to check the previous signal)
#         if len(candles) < 2:
#             print("Not enough data to make a trade decision.")
#             return

#         # Get the signal from the row before the one currently forming
#         sim_previous_signal = candles.iloc[-2]['signal']

#         if sim_previous_signal == 1:
#             # If signal is 1 and no position is open, simulate going long (buy)
#             if not sim_position_open:
#                 # Simulate buy entry
#                 sim_entry_price = candles.iloc[-2]['close']
#                 sim_position_open = True
                
#                 # Record simulated signal and entry price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = 1
#                 candles.at[candles.index[-2], 'sim_entry'] = sim_entry_price

#                 print(f"Simulated buy at {sim_entry_price}")

#             else:
#                 print("Position already open, no action taken.")

#         elif sim_previous_signal == 0 or sim_previous_signal == -1:
#             # If signal is 0 or -1 and a position is open, simulate closing the position (sell)
#             if sim_position_open:
#                 # Simulate sell exit
#                 sim_exit_price = candles.iloc[-2]['close']
#                 sim_position_open = False

#                 # Record simulated signal and exit price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = -1
#                 candles.at[candles.index[-2], 'sim_exit'] = sim_exit_price

#                 print(f"Simulated sell at {sim_exit_price}")
#             else:
#                 print("No position open, no action taken.")

# def trade_simulator(candles):
#     """
#     Simulated function to decide whether to go long or close positions based on the signal column.

#     Parameters:
#     - candles (pd.DataFrame): The DataFrame containing minute candle data.

#     The function updates the 'sim_signal' and 'sim_price' columns in the DataFrame.
#     """

#     global sim_position_open, sim_entry_price  # Ensure access to global variables

#     current_time = datetime.now(pytz.timezone('US/Eastern'))
#     current_hour = datetime.now(pytz.timezone('US/Eastern')).hour    
#     current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
#     current_second = datetime.now(pytz.timezone('US/Eastern')).second

#     # Initialize the new columns if they don't exist
#     if 'sim_signal' not in candles.columns:
#         candles['sim_signal'] = None
#     if 'sim_price' not in candles.columns:
#         candles['sim_price'] = None

#     # Check if it's the start of a new minute (second == 00)
#     if current_time.second == 0:
#         # Ensure there is enough data in candles (at least 2 rows to check the previous signal)
#         if len(candles) < 2:
#             print("Not enough data to make a trade decision.")
#             return

#         # Get the signal from the row before the one currently forming
#         sim_previous_signal = candles.iloc[-2]['signal']

#         if sim_previous_signal == 1:
#             # If signal is 1 and no position is open, simulate going long (buy)
#             if not sim_position_open:
#                 # Simulate buy entry
#                 sim_entry_price = candles.iloc[-2]['close']
#                 sim_position_open = True
                
#                 # Record simulated signal and entry price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = 1
#                 candles.at[candles.index[-2], 'sim_price'] = sim_entry_price

#                 print(f"Simulated buy at {sim_entry_price}")

#             else:
#                 print("Position already open, no action taken.")

#         elif sim_previous_signal == 0 or sim_previous_signal == -1:
#             # If signal is 0 or -1 and a position is open, simulate closing the position (sell)
#             if sim_position_open:
#                 # Simulate sell exit
#                 sim_exit_price = candles.iloc[-2]['close']
#                 sim_position_open = False

#                 # Record simulated signal and exit price in the DataFrame
#                 candles.at[candles.index[-2], 'sim_signal'] = -1
#                 candles.at[candles.index[-2], 'sim_price'] = sim_exit_price

#                 print(f"Simulated sell at {sim_exit_price}")
#             else:
#                 print("No position open, no action taken.")

# <h1 style="color:red;"><b>The Data Handler<b></h1>

In [ ]:
linked_accounts = client.account_linked().json()
linked_accounts

account_hash = linked_accounts[0].get('hashValue')
print(account_hash)

# equity_ticker = 'MARA'
futures_ticker = '/NQ'

buy_order = {"orderType": "MARKET",
        "session": "NORMAL",
        "duration": "DAY",
        "orderStrategyType": "SINGLE",
        "orderLegCollection": [
            {
                "instruction": "BUY",
                "quantity": 1,
                "instrument": {
                    "symbol": futures_ticker,
                    "assetType": "EQUITY"
                }
            }
            ]
        }  

sell_order = {"orderType": "MARKET",
        "session": "NORMAL",
        "duration": "DAY",
        "orderStrategyType": "SINGLE",
        "orderLegCollection": [
            {
                "instruction": "SELL",
                "quantity": 1,
                "instrument": {
                    "symbol": futures_ticker,
                    "assetType": "EQUITY"
                }
            }
            ]
        }  

In [ ]:
streamer = client.stream

In [ ]:
tickers = "/ES,/NQ,/CL,/GC"
# tickers = "MARA,PLUG,SOFI,SWN,RKLB"

ticker_to_point_value = {
    "/ES": 50,       # E-Mini S&P 500
    "/NQ": 20,       # E-Mini NASDAQ 100
    "/CL": 1000,     # Crude Oil
    "/GC": 100,      # Gold
}

ticker_to_tick_size = {
    "/ES": 0.25,        # E-Mini S&P 500
    "/NQ": 0.25,        # E-Mini NASDAQ 100
    "/CL": 0.01,        # Crude Oil
    "/GC": 0.1,         # Gold
}

ma_periods = [1, 3, 5]
sma_periods = ma_periods
wma_periods = ma_periods
rsi_periods = ma_periods

ma_combinations = [
    (f'wma_{wma}', f'sma_{sma}', f'rsi_{wma}')
    for wma in wma_periods
    for sma in sma_periods
    if wma <= sma
]

compression_factors = range(1, 6, 2)

def handle_data(message):
    global minute_candles

    current_time_USE = datetime.now(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d %H:%M:%S')
    current_hour = datetime.now(pytz.timezone('US/Eastern')).hour
    current_minute = datetime.now(pytz.timezone('US/Eastern')).minute
    current_second = datetime.now(pytz.timezone('US/Eastern')).second

    # Stop the streamer at 16:59:55
    if current_hour == 16 and current_minute == 59 and current_second >= 55:
        print(f"{current_time_USE.strftime('%Y-%m-%d %H:%M:%S')} - Stopping the streamer.")
        streamer.stop(clear_subscriptions=True)
        return  # Exit the function immediately after stopping the streamer

    # Try parsing the incoming message
    try: # Indent the code and uncomment the try-except block to revert
        data = json.loads(message)

        # Call the function to process the data and update todays_price_action
        process_message_data(data)

        # **Update the minute candles in real-time**
        update_minute_candles(ticker_tables, time_frame='min')

        # Compress each ticker in minute_candles in real-time
        for ticker, df in minute_candles.items():
            for compression_factor in compression_factors:
                dynamic_name = f"{compression_factor}_minute_compression"
                compressed_sessions[ticker][dynamic_name] = compress_candles(df, compression_factor)


        # Iterate through all the tickers in minute_candles
        for ticker, compressions in compressed_sessions.items():
            for compression_name, df in compressions.items():
                    # Calculate moving averages and indicators for each DataFrame
                    compressed_sessions[ticker][compression_name] = calculate_indicators_1(
                        df, 
                        price_col='close', 
                        acc_vol_col='accumulative_volume', 
                        sma_periods=sma_periods,
                        wma_periods=wma_periods,
                        rsi_periods=rsi_periods,
                        candle_window=10
                    )
        
        # ANY TRADE STRATEGY LOGIC IS TO GO BETWEEN THE LINES OF ASTERISKS
        # *********************************************************************************************************************************************************************************
        strategy_types = ['trend', 'reversion']
        order_types = ['market', 'limit']
        directional_biases = ['long', 'short']
        directional_bias = 'long'

        # Iterate through all tickers in compressed_sessions
        for ticker, sessions in compressed_sessions.items():
            print(f"Processing ticker: {ticker}")

            # Iterate through all sessions of the current ticker
            for time_slice, compressions in sessions.items():
                print(f"  {ticker} Processing session: {time_slice}, Total Compressions: {len(compressions)}")

                # Iterate through all compression levels of the current session
                for compression_name, df in compressions.items():
                    print(f"    {ticker} Processing compression: {compression_name} in session {time_slice}")

                    # Iterate through all the ma_combinations
                    for sig_ma, con_ma, rsi_col in ma_combinations:

                        for strategy_type in strategy_types:
                            print(f"      {ticker} {time_slice} {compression_name} Applying {strategy_type} strategy for {sig_ma} and {con_ma}")
                            
                            for order_type in order_types:
                                print(f"        {ticker} Applying {order_type} order_type")

                                # for directional_bias in directional_biases:
                                #     print(f"          Applying {directional_bias} directional bias")

                                # Generate trading signals for the current strategy
                                compressions[compression_name] = generate_trading_signals_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Update position_open columns to be 1:1 verbal boolean with the signal
                                compressions[compression_name] = update_position_open_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Determine entry prices for each ticker
                                compressions[compression_name] = determine_entry_prices_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    ticker_to_tick_size=ticker_to_tick_size,
                                    ticker=ticker,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Determine exit prices for each ticker
                                compressions[compression_name] = determine_exit_prices_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    ticker_to_tick_size=ticker_to_tick_size,
                                    ticker=ticker,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Stop loss calculation
                                compressions[compression_name] = calculate_stop_losses_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Track stop loss hits
                                compressions[compression_name] = track_stop_loss_hits_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    ticker_to_tick_size=ticker_to_tick_size,
                                    ticker=ticker,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Adjust signals from stop loss hits
                                compressions[compression_name] = adjust_signals_for_stop_loss_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Re-update position_open column after stop loss hits
                                compressions[compression_name] = update_position_open_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Re-determine entry prices after stop loss hits
                                compressions[compression_name] = determine_entry_prices_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    ticker_to_tick_size=ticker_to_tick_size,
                                    ticker=ticker,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Re-determine exit prices after stop loss hits
                                compressions[compression_name] = determine_exit_prices_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    ticker_to_tick_size=ticker_to_tick_size,
                                    ticker=ticker,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                )

                                # Update stop loss levels after stop loss hits
                                compressions[compression_name] = update_stop_loss_1(
                                    df,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias
                                    
                                )

                                # Calculate profit/loss for each ticker's DataFrame
                                compressions[compression_name] = calculate_profit_loss_1(
                                    df,
                                    contract_multiplier=1,
                                    ma_name1=sig_ma,
                                    ma_name2=con_ma,
                                    rsi_column=rsi_col,
                                    strategy_type=strategy_type,
                                    order_type=order_type,
                                    directional_bias=directional_bias   
                                )
        # *********************************************************************************************************************************************************************************        
        # ANY TRADE STRATEGY LOGIC IS TO GO BETWEEN THE LINES OF ASTERISKS

        if current_second == 59 or 00 or 1 or 2:
            # Call the trade_logic function to execute trades based on the signal
            trade_logic(minute_candles, buy_order=buy_order, sell_order=sell_order)

        if current_second == 59 or 00 or 1 or 2:
            # Call the trade_simulator function to simulate trades based on the signal
            trade_simulator(minute_candles)

            print(message)

    except Exception as e:
        print(f"{current_time_USE}: Error processing message: {e}")

Suppress all warnings to avoid too much printing under the streamer cell.

In [ ]:
warnings.filterwarnings("ignore")

Start the streamer and send the level one futures subscription

In [ ]:
streamer.start(handle_data)
streamer.send(streamer.level_one_futures(tickers, "0,1,2,3,4,5,8,9,12,13,14,18,19,20,23,25,26,35,37,38")) # For futures
# streamer.send(streamer.level_one_equities(tickers, "0,1,2,3,4,5,8,9,10,11,12,17,18,19,20,21,42")) # For equities

Stop the streamer

In [ ]:
streamer.stop(clear_subscriptions=True)

Append new data to the csv storage files that contain the raw, manually collected OHLC data from Schwab

In [ ]:
def save_dfs_to_csv(candles, save_path="raw_futures_data/", columns_to_save=None):
    """ After specifying column data types
    Save each DataFrame in the dictionary to a separate CSV file, appending to the CSV
    if the file already exists, saving only specified columns with enforced data types.

    Parameters:
    - candles (dict): A dictionary where keys are names and values are DataFrames.
    - save_path (str): The directory where the CSV files will be saved. Default is 'dataframes/'.
    - columns_to_save (list): A list of columns to save from each DataFrame. Default is None (save all columns).
    """
    # Ensure the directory exists
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Define column data types
    dtype_conversion = {
        'open': 'float64',
        'high': 'float64',
        'low': 'float64',
        'close': 'float64',
        'accumulative_volume': 'float64'
    }

    # Save each DataFrame
    for key, df in candles.items():
        file_name = os.path.join(save_path, f"{key.replace("/", "")}.csv")

        # Filter the DataFrame to only the specified columns, if provided
        if columns_to_save:
            df_to_save = df[columns_to_save].copy()
        else:
            df_to_save = df.copy()

        # Ensure correct data types for numeric columns
        for col, dtype in dtype_conversion.items():
            if col in df_to_save.columns:
                df_to_save[col] = df_to_save[col].astype(dtype, errors='ignore')

        # Save the DataFrame
        if os.path.exists(file_name):
            df_to_save.to_csv(file_name, mode='a', header=False, index=False)
            print(f"Appended to: {file_name}")
        else:
            df_to_save.to_csv(file_name, index=False)
            print(f"Saved new file: {file_name}")

columns_to_save = ['datetime', 'ticker', 'open', 'high', 'low', 'close', 'accumulative_volume']
save_dfs_to_csv(minute_candles, save_path="raw_futures_data/", columns_to_save=columns_to_save)

Appended to: raw_futures_data/ES.csv
Appended to: raw_futures_data/NQ.csv
Appended to: raw_futures_data/CL.csv
Appended to: raw_futures_data/GC.csv


Inspect data from the current session. Check current time to see if the streamer has been cut off. Restart the streamer if it has

In [ ]:
key = 1

tickers_list = list(ticker_tables.keys())
display(minute_candles)
display(tickers_list)
display(ticker_tables[tickers_list[key]])
display(minute_candles[tickers_list[key]])

{'/ES':                       datetime ticker       open       high        low  \
 0    2025-05-12 18:12:00-04:00    /ES 5,864.0000 5,864.7500 5,863.7500   
 1    2025-05-12 18:13:00-04:00    /ES 5,864.5000 5,864.5000 5,863.0000   
 2    2025-05-12 18:14:00-04:00    /ES 5,863.2500 5,863.2500 5,863.0000   
 3    2025-05-12 18:15:00-04:00    /ES 5,863.2500 5,863.5000 5,862.5000   
 4    2025-05-12 18:16:00-04:00    /ES 5,863.0000 5,864.0000 5,862.0000   
 ...                        ...    ...        ...        ...        ...   
 1290 2025-05-13 15:42:00-04:00    /ES 5,921.7500 5,924.0000 5,921.7500   
 1291 2025-05-13 15:43:00-04:00    /ES 5,923.7500 5,924.0000 5,922.2500   
 1292 2025-05-13 15:44:00-04:00    /ES 5,922.5000 5,923.0000 5,922.0000   
 1293 2025-05-13 15:45:00-04:00    /ES 5,922.2500 5,922.7500 5,920.0000   
 1294 2025-05-13 15:46:00-04:00    /ES 5,920.2500 5,921.5000 5,919.7500   
 
           close accumulative_volume  
 0    5,864.5000                2454  
 1    5,863.2

['/ES', '/NQ', '/CL', '/GC']

,timestamp,key,bid_price,ask_price,last_price,bid_size,ask_size,total_volume,last_size,high_price,low_price,close_price,open_price,net_change,future_pct_change,open_interest,tick,tick_amount,future_exp_date,ask_time,bid_time
datetime,,,,,,,,,,,,,,,,,,,,,
2025-05-12 18:12:02.003000-04:00,1747087922003,/NQ,"20,937.7500","20,939.7500","20,938.5000",1,1,1344,1,"20,963.5000","20,931.7500","20,136.7500","20,954.7500",801.7500,3.9815,259062,0.2500,5,1750392000000,1747087921300,1747087921227
2025-05-12 18:12:03.026000-04:00,1747087923026,/NQ,"20,939.5000","20,941.7500","20,940.7500",None,4,1350,0,NaN,NaN,NaN,NaN,804.0000,3.9927,None,NaN,None,None,1747087922568,1747087922893
2025-05-12 18:12:04.029000-04:00,1747087924029,/NQ,"20,939.7500",NaN,"20,941.7500",None,3,1351,0,NaN,NaN,NaN,NaN,805.0000,3.9977,None,NaN,None,None,1747087923256,1747087923707
2025-05-12 18:12:07.138000-04:00,1747087927138,/NQ,NaN,NaN,"20,941.7500",3,None,None,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,1747087926812
2025-05-12 18:12:08.161000-04:00,1747087928161,/NQ,"20,940.7500","20,942.7500","20,942.5000",1,1,1362,0,NaN,NaN,NaN,NaN,805.7500,4.0014,None,NaN,None,None,1747087927253,1747087927281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-13 15:46:54.415000-04:00,1747165614415,/NQ,"21,314.5000",NaN,"21,314.5000",1,4,467548,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,1747165614255,1747165614255
2025-05-13 15:46:55.422000-04:00,1747165615422,/NQ,"21,316.0000","21,316.5000","21,316.2500",3,2,467554,0,NaN,NaN,NaN,NaN,367.5000,1.7543,None,NaN,None,None,1747165615257,1747165615177
2025-05-13 15:46:56.431000-04:00,1747165616431,/NQ,NaN,NaN,"21,316.0000",1,4,467560,0,NaN,NaN,NaN,NaN,367.2500,1.7531,None,NaN,None,None,1747165616263,1747165616263


,datetime,ticker,open,high,low,close,accumulative_volume
0,2025-05-12 18:12:00-04:00,/NQ,"20,938.5000","20,942.5000","20,938.5000","20,942.0000",1398
1,2025-05-12 18:13:00-04:00,/NQ,"20,942.0000","20,942.0000","20,934.7500","20,936.0000",1446
2,2025-05-12 18:14:00-04:00,/NQ,"20,936.0000","20,936.0000","20,932.5000","20,933.5000",1534
3,2025-05-12 18:15:00-04:00,/NQ,"20,933.5000","20,936.2500","20,932.2500","20,934.2500",1646
4,2025-05-12 18:16:00-04:00,/NQ,"20,933.5000","20,939.2500","20,927.7500","20,939.2500",1831
...,...,...,...,...,...,...,...
1290,2025-05-13 15:42:00-04:00,/NQ,"21,319.5000","21,330.0000","21,319.5000","21,330.0000",464744
1291,2025-05-13 15:43:00-04:00,/NQ,"21,329.2500","21,329.5000","21,321.7500","21,323.2500",465291
1292,2025-05-13 15:44:00-04:00,/NQ,"21,322.0000","21,324.0000","21,320.0000","21,322.5000",465782
1293,2025-05-13 15:45:00-04:00,/NQ,"21,322.5000","21,323.0000","21,311.7500","21,315.0000",466716
